# Setup

Start by importing TF Encrypted. We don't need to import TensorFlow as well but it's often very convenient since we can mix ordinary and encrypted computations.

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

from utils import print_in_notebook

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/vladimirpopov/ML/tf-encrypted/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


We only need the following step since we want to inspect the computation later in TensorBoard. It should normally be skipped to avoid the implied overhead of generating event and tracing files.

In [3]:
%load_ext tensorboard

TENSORBOARD_DIR = "/tmp/tensorboard"

tfe.set_tfe_trace_flag(True)
tfe.set_tfe_events_flag(True)
tfe.set_log_directory(TENSORBOARD_DIR)

# Computation

We next define our mixed computation, in this case summing two encrypted (i.e. private) tensors coming from different input providers. Note that we are using ordinary TensorFlow to generate the inputs locally on the providers.

In [4]:
x = tfe.define_private_input("input-provider-x", lambda: tf.fill([2,2], 2))
y = tfe.define_private_input("input-provider-y", lambda: tf.fill([2,2], 3))

z = x + y

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




At this point `z` contains the encrypted sum. To reveal only the sum to a result receiver you would normally use the following, which decrypts and executes the print function locally on the result receiver:

In [8]:
compute_op = tfe.define_output("result-receiver", z, tf.print)

However, since we are running in a notebook the above wouldn't actually display anything. To get around this we can use the `print_in_notebook` function from `utils` instead.

We stress that this is only because we are running in a notebook, and using `py_func` is for instance not possible when running in an actual distributed execution context. See the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/print) for more information.

In [6]:
compute_op = tfe.define_output("result-receiver", z, print_in_notebook)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


# Execution

Having defined our computation we use a `tfe.Session` to run it, optionally passing in a tag when we want event and tracing files to be written.

Here we first remove previous event and tracing files to make it easier to find the new runs in TensorBoard.

In [9]:
!rm -rf {TENSORBOARD_DIR}

with tfe.Session() as sess:
    sess.run(compute_op, tag='sum')
    sess.run(compute_op, tag='sum')

[[5 5]
 [5 5]]
[[5 5]
 [5 5]]


# Inspection

We can finally inspect our computations in TensorBoard using the tags passed to `sess.run`.

Note that this is not saved in notebooks so nothing will show below unless you run this yourself.

In [10]:
%tensorboard --logdir {TENSORBOARD_DIR}

Launching TensorBoard...